# Lens Maker Equation

Here we create a lens with a fixed shape and compute what its focal length should come out to, using the formula known as the Lens Maker Equation. Then, we use torchlensmaker forward pass to compute the loss and check that it is close to zero.

Note that it will not be exactly zero because the lens maker equation is an approximation.

In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

from torchlensmaker.raytracing import super_refraction, clamped_refraction, ray_point_squared_distance, refraction
from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack, Anchor
from torchlensmaker.shapes import CircularArc
from torchlensmaker.training import render, optimize

In [ ]:
# Lens diameter: 40mm
lens_diameter = 40.0

# Circular surface radius
R1, R2 = -200, 250

# Indices of refraction
N1, N2 = (1.0, 1.50)

# Lens Maker Equation
expected_focal_length = - 1 / (
    ((N2 - N1) / N1) * (1/R1 - 1/R2)
)

print("Expected focal length (lens's maker equation):", expected_focal_length)

# Surface shapes of the lens
surface1 = CircularArc(lens_diameter, R1)
surface2 = CircularArc(lens_diameter, R2)

# Setup the optical stack with incident parallel rays
optics = OpticalStack([
    ParallelBeamRandom(radius=lens_diameter),
    FixedGap(10.),
    
    RefractiveSurface(surface1, (N1, N2), anchors=(Anchor.Center, Anchor.Edge)),
    FixedGap(10.),
    RefractiveSurface(surface2, (N2, N1), anchors=(Anchor.Edge, Anchor.Center)),
    
    FixedGap(expected_focal_length - 5.),  # is focal length wrt to the first surface?
    FocalPointLoss(),
])


render(optics, num_rays=10)

# Compute loss with 20 rays
loss = optics.forward(20).item()
print("Loss:", loss)


# Next

* optimize the gap with VariableGap to improve the best fit focal distance estimation
* resample the shape to a parabola to reduce spherical aberation